In [1]:
!pip -q install langchain-groq
!pip -q install -U langchain_community tiktoken langchainhub
!pip -q install -U langchain langgraph tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7

In [2]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [3]:
### Tracing (optional)
import os
from pprint import pprint

# os.environ['LANGCHAIN TRACING V2'] = 'true'
# os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
# os.environ['LANGCHAIN API KEY'] = userdata.get('LANGCHAIN_API_KEY')

In [4]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            model="llama3-70b-8192",
        )

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

In [6]:
from langchain.schema import Document
from langgraph.graph import END, StateGraph

In [7]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=1)

In [8]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in understanding financial trading preferences based on user inputs and extracting key information. Your task is to read the USER_MESSAGE and identify the following elements:
- stock_ticker: The stock symbol the user is interested in (if specified).
- risk_tolerance: The user's level of risk tolerance (e.g., Very Low, Low, Medium, High, Very High).
- trading_strategy: The user's preferred trading strategy (e.g., Long-Term, Short-Term, Swing, Day Trading etc.).

If any of the above elements are not specified in the user message, return null for that element.

Provide a JSON object with a single key 'categorized_results' that contains the extracted 'stock_ticker', 'risk_tolerance', and 'trading_strategy' elements. This means your task is to return a single-key JSON object containing the 'categorized_results' key. Inside the 'categorized_results' key, there should be the keys 'stock', 'risk_tolerance', and 'trading_strategy' with the values you identified in return. Do not add any preamble or explanation beyond this JSON object.<|eot_id|><|start_header_id|>user<|end_header_id|>

USER_MESSAGE:\n\n{user_message}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["user_message"],
)

json_generator = prompt | GROQ_LLM | JsonOutputParser()

user_message = """I am interested in apple stock. I don't want to take a big risk but if it will benefit me in the long run, I am willing to take some risks."""

categorized_results = json_generator.invoke({"user_message": user_message})

print(categorized_results)

{'categorized_results': {'stock': 'AAPL', 'risk_tolerance': 'Medium', 'trading_strategy': 'Long-Term'}}


In [9]:
search_keyword_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

As an expert in financial market data retrieval, your task is to determine the best search prompts for gathering relevant information based on the user inputs. These search prompts will be used to collect data necessary for subsequent financial analysis.

Given the USER_MESSAGE and CATEGORIZED_RESULTS which include STOCK_TICKER, RISK_TOLERANCE, and TRADING_STRATEGY, identify the search prompts that will yield the most relevant market data. Each search prompt should aim to gather specific information that will aid in financial analysis.

You should look for:
1. Current stock price and historical price trends for STOCK_TICKER.
2. Market trend predictions related to STOCK_TICKER considering the specified TRADING_STRATEGY. For example, if the specified TRADING_STRATEGY is 'Day Trading', look for the price predictions of STOCK_TICKER over 24 hours. If the specified TRADING_STRATEGY is 'Long-Term Investment', look for the price predictions of STOCK_TICKER over the long term.
3. Analysis and predictions from financial analysts for STOCK_TICKER.
4. News and recent developments affecting STOCK_TICKER.
5. Risk assessment and management strategies relevant to the specified RISK_TOLERANCE and TRADING_STRATEGY.

Provide a JSON object with a single key 'search_prompts', containing a total of 5 prompts. Do not include any preamble or explanation.<|eot_id|><|start_header_id|>user<|end_header_id|>

USER_MESSAGE: {user_message}

CATEGORIZED_RESULTS: {categorized_results}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["user_message", "categorized_results"],
)

search_keyword_chain = search_keyword_prompt | GROQ_LLM | JsonOutputParser()

search_prompts = search_keyword_chain.invoke({"user_message": user_message, "categorized_results": categorized_results})

print(search_prompts)

{'search_prompts': ['Current and historical stock prices of AAPL', 'Long-term price predictions of AAPL based on technical analysis and market trends', 'AAPL stock forecast and analysis from reputable financial analysts', 'Recent news and developments affecting AAPL stock performance', 'Medium risk tolerance investment strategies for long-term AAPL stock holdings']}


In [10]:
# Web search
full_searches = []
for keyword in search_prompts['search_prompts']:
    print(keyword)
    temp_docs = web_search_tool.invoke({"query": keyword})
    web_results = "\n".join([d["content"] for d in temp_docs])
    web_results = Document(page_content=web_results)
    if full_searches is not None:
        full_searches.append(web_results)
    else:
        full_searches = [web_results]
print(full_searches)
print(type(full_searches))

Current and historical stock prices of AAPL
Long-term price predictions of AAPL based on technical analysis and market trends
AAPL stock forecast and analysis from reputable financial analysts
Recent news and developments affecting AAPL stock performance
Medium risk tolerance investment strategies for long-term AAPL stock holdings
[Document(metadata={}, page_content='Historical daily share price chart and data for Apple since 1980 adjusted for splits and dividends. The latest closing stock price for Apple as of September 26, 2024 is 227.52.. The all-time high Apple stock closing price was 234.55 on July 16, 2024.; The Apple 52-week high stock price is 237.23, which is 4.3% above the current share price.; The Apple 52-week low stock price is 164.07, which ...\n170.69. 169.82. 56,294,400. Discover historical prices for AAPL stock on Yahoo Finance. View daily, weekly or monthly format back to when Apple Inc. stock was issued.\n171.96. 171.08. 64,588,900. Discover historical prices for AAP

In [11]:
strategy_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in financial strategy analysis. Your task is to evaluate the feasibility and potential profitability of a specified trading strategy for a given stock, considering the user's risk tolerance and the provided research information.

Given the USER_MESSAGE, CATEGORIZED_RESULTS (which include STOCK_TICKER, RISK_TOLERANCE, and TRADING_STRATEGY), and RESEARCH_INFO, assess the following:
- How suitable is the specified trading strategy (TRADING_STRATEGY) for the given stock (STOCK_TICKER)?
- What are the potential profitability and risks associated with this strategy, given the user's risk tolerance (RISK_TOLERANCE)?
- Consider historical performance, technical indicators, and trading signals from the research information (RESEARCH_INFO).

Provide a comprehensive and detailed evaluation and recommendation on whether to proceed with the trading strategy. Your analysis should include:

1. **Suitability**:
   - Assess the suitability of the TRADING_STRATEGY for STOCK_TICKER, considering the user's RISK_TOLERANCE and RESEARCH_INFO.

2. **Potential Profitability**:
   - Evaluate the potential profitability of STOCK_TICKER if the TRADING_STRATEGY is applied, considering the user's RISK_TOLERANCE and RESEARCH_INFO.

3. **Risks**:
   - Analyze the risks associated with STOCK_TICKER when applying the TRADING_STRATEGY, considering RESEARCH_INFO. Evaluate if these risks align with the user's RISK_TOLERANCE.

4. **Recommendations**:
   - Based on your analysis, provide recommendations:
     - If the user's RISK_TOLERANCE is low and the projected risks are medium or high, recommend against this investment.
     - If the user's RISK_TOLERANCE is medium and the projected risks are medium, advise the user to proceed with caution.
     - If the user's RISK_TOLERANCE is high and the projected risks are medium or low, recommend proceeding with the investment.
     - For other scenarios, use similar logic to shape your recommendation.

5. **Current and Projected Prices**:
   - Include the current price of STOCK_TICKER.
   - Provide projected price predictions based on the TRADING_STRATEGY:
     - For 'Day Trading', provide predictions for the next 24 hours.
     - For 'Long-Term Investment', provide long-term price predictions.
     - For other scenarios, use similar logic to shape your predictions.
     - If confident predictions cannot be made, explain why.

6. **Strategy Explanation**:
   - If recommending to proceed, explain the strategy in depth, including specific steps and considerations.
   - If not recommending to invest, provide a detailed explanation of the reasons and potential risks.

Be thorough and specific in your evaluation, avoiding general statements. Tailor your analysis to the specifics of the stock, strategy, and user's risk tolerance. Remember that you are an expert in analysing and developing financial strategies. You must be confident about your answers. You are the decision maker here.

Return your findings as a JSON object with a single key 'strategy_analysis' containing your evaluation and recommendation. Do not include any preamble or explanation apart from this JSON object.<|eot_id|><|start_header_id|>user<|end_header_id|>

USER_MESSAGE: {user_message}

CATEGORIZED_RESULTS: {categorized_results}

RESEARCH_INFO: {research_info}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["user_message","categorized_results","research_info"],
)

strategy_chain = strategy_prompt | GROQ_LLM | JsonOutputParser()

strategy_analysis = strategy_chain.invoke({"user_message": user_message, "categorized_results": categorized_results, "research_info": full_searches})

print(strategy_analysis)

{'strategy_analysis': {' Suitability': "The specified long-term trading strategy is suitable for Apple Inc. (AAPL) stock, considering the user's medium risk tolerance. AAPL has historically demonstrated a capacity for robust growth, and the current sentiment is neutral, suggesting a balanced market without extreme buying or selling pressure.", 'Potential Profitability': 'Based on the research information, the potential profitability of AAPL stock with the long-term trading strategy is moderate to high. The average target price of $238.76 and $241.63, respectively, from the 36 and 30 analysts, indicates an increase of 4.94% and 6.20% from the current stock price of $227.52.', 'Risks': 'The risks associated with AAPL stock when applying the long-term trading strategy are medium. The stock has demonstrated robust growth, but there are potential risks from fluctuating operating income and expenses, as well as global economic factors. However, the overall sentiment is neutral, and the compa

In [12]:
risk_assessment_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in financial risk assessment. Your task is to evaluate the risks associated with the specified trading strategy for the given stock, considering the user's risk tolerance, the research information, and the strategy analysis provided.

Given the USER_MESSAGE, CATEGORIZED_RESULTS which includes STOCK_TICKER, RISK_TOLERANCE, and TRADING_STRATEGY, RESEARCH_INFO, and STRATEGY_ANALYSIS, assess the following:
- Identify and evaluate specific risks associated with the trading strategy (TRADING_STRATEGY) for the stock (STOCK_TICKER).
- Consider market volatility, potential loss, historical risk factors, and any external factors mentioned in the research information (RESEARCH_INFO).
- Assess how these risks align with the user's risk tolerance (RISK_TOLERANCE).
- Use the detailed findings from the strategy analysis (STRATEGY_ANALYSIS) to inform your risk evaluation.

Provide a comprehensive and detailed risk assessment that includes:
1. Identification of specific risks based on RESEARCH_INFO and STRATEGY_ANALYSIS.
2. The potential financial impact of these risks, using precise data from the research.
3. An assessment of how these risks align with the user's risk tolerance (RISK_TOLERANCE).
4. A final risk assessment conclusion, deciding whether to proceed with the trading strategy or not, with clear and data-backed reasoning.

Your analysis should be conclusive, clearly deciding whether the trading strategy is suitable based on the risks and the user's tolerance. Avoid general advice or vague recommendations, and provide a clear decision and explanation based on the provided data.

Return your findings as a JSON object with a single key 'risk_assessment' containing your detailed risk evaluation. Do not include any preamble or explanation apart from this JSON object.<|eot_id|><|start_header_id|>user<|end_header_id|>

USER_MESSAGE: {user_message}

CATEGORIZED_RESULTS: {categorized_results}

RESEARCH_INFO: {research_info}

STRATEGY_ANALYSIS: {strategy_analysis}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["user_message","categorized_results","research_info","strategy_analysis"],
)

risk_assessment_chain = risk_assessment_prompt | GROQ_LLM | JsonOutputParser()

risk_assessment = risk_assessment_chain.invoke({"user_message": user_message, "categorized_results": categorized_results, "research_info": full_searches, "strategy_analysis": strategy_analysis})

print(risk_assessment)

{'risk_assessment': "Based on the analysis, I conclude that the long-term trading strategy for Apple Inc. (AAPL) stock is suitable for the user, considering their medium risk tolerance. The potential profitability of AAPL stock with this strategy is moderate to high, with an average target price of $238.76 and $241.63, respectively, from the 36 and 30 analysts. The risks associated with AAPL stock are medium, mainly due to fluctuating operating income and expenses, as well as global economic factors. However, the overall sentiment is neutral, and the company has demonstrated a capacity for adapting to changing market conditions.\n\nThe user's medium risk tolerance aligns with the moderate to high potential profitability and medium risks associated with the stock. Therefore, I recommend proceeding with the long-term trading strategy for AAPL stock. It is essential to maintain a diversified portfolio and regularly review the stock's performance to ensure alignment with the user's risk to

In [13]:
sentiment_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in financial sentiment analysis. Your task is to evaluate the overall sentiment regarding the specified trading strategy for the given stock, considering the user's risk tolerance, the research information, the strategy analysis, and the risk assessment provided.

Given the USER_MESSAGE, CATEGORIZED_RESULTS which includes STOCK_TICKER, RISK_TOLERANCE, and TRADING_STRATEGY, RESEARCH_INFO, STRATEGY_ANALYSIS, and RISK_ASSESSMENT, assess the following:
- Determine the overall sentiment (positive or negative) towards proceeding with the trading strategy.
- Base your evaluation on the detailed findings from the research information (RESEARCH_INFO), the strategy analysis (STRATEGY_ANALYSIS), and the risk assessment (RISK_ASSESSMENT).
- Provide a clear and concise explanation for your sentiment decision, referencing specific points from the inputs to justify your conclusion.

Your analysis should be conclusive and clearly state the overall sentiment. Avoid general advice or ambiguous recommendations. Provide a clear sentiment decision with a detailed explanation based on the provided data. You are the decision maker here.

Return your findings as a JSON object with a single key 'sentiment_analysis' containing two fields: 'sentiment' and 'explanation'. Do not include any preamble or explanation apart from this JSON object.<|eot_id|><|start_header_id|>user<|end_header_id|>

USER_MESSAGE: {user_message}

CATEGORIZED_RESULTS: {categorized_results}

RESEARCH_INFO: {research_info}

STRATEGY_ANALYSIS: {strategy_analysis}

RISK_ASSESSMENT: {risk_assessment}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["user_message","categorized_results","research_info","strategy_analysis","risk_assessment"],
)

sentiment_chain = sentiment_prompt | GROQ_LLM | JsonOutputParser()

sentiment_analysis = sentiment_chain.invoke({"user_message": user_message, "categorized_results": categorized_results, "research_info": full_searches, "strategy_analysis": strategy_analysis, "risk_assessment": risk_assessment})

print(sentiment_analysis)

{'sentiment_analysis': {'sentiment': 'Positive', 'explanation': "Based on the research information, strategy analysis, and risk assessment, I conclude that the overall sentiment towards proceeding with the long-term trading strategy for Apple Inc. (AAPL) stock is positive. The strategy is suitable for the user's medium risk tolerance, and the potential profitability is moderate to high. Although there are medium risks associated with AAPL stock, the company has demonstrated a capacity for robust growth and adapting to changing market conditions. The user's medium risk tolerance aligns with the potential profitability and risks associated with the stock. Therefore, I recommend proceeding with the long-term trading strategy for AAPL stock, while maintaining a diversified portfolio and regularly reviewing the stock's performance to ensure alignment with the user's risk tolerance and investment goals."}}


In [14]:
decision_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are the Decision Agent, an expert in financial decision-making. Your task is to make a final decision on whether to proceed with the trading strategy for the given stock, considering all the previous inputs.

Given the USER_MESSAGE, CATEGORIZED_RESULTS which includes STOCK_TICKER, RISK_TOLERANCE, and TRADING_STRATEGY, RESEARCH_INFO, STRATEGY_ANALYSIS, RISK_ASSESSMENT, and SENTIMENT_ANALYSIS, assess the following:
- Make a confident final decision on whether to proceed with the trading strategy.
- Base your decision on the detailed findings from the research information (RESEARCH_INFO), the strategy analysis (STRATEGY_ANALYSIS), the risk assessment (RISK_ASSESSMENT), and the sentiment analysis (SENTIMENT_ANALYSIS).
- Provide a concise explanation for your decision, referencing specific points from the inputs to justify your conclusion.

Your decision should be definitive and well-justified. Do not provide ambiguous recommendations. Make a clear decision with a concise and confident explanation based on the provided data. Remember, you are the decision-maker here. You should be confident about your decision.

Return your findings as a JSON object with a single key 'final_decision' containing two fields: 'decision' (whether "proceed" or "do not proceed") and 'explanation'. Do not include any preamble or explanation apart from this JSON object.<|eot_id|><|start_header_id|>user<|end_header_id|>

USER_MESSAGE: {user_message}

CATEGORIZED_RESULTS: {categorized_results}

RESEARCH_INFO: {research_info}

STRATEGY_ANALYSIS: {strategy_analysis}

RISK_ASSESSMENT: {risk_assessment}

SENTIMENT_ANALYSIS: {sentiment_analysis}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["user_message","categorized_results","research_info","strategy_analysis","risk_assessment", "sentiment_analysis"],
)

decision_chain = decision_prompt | GROQ_LLM | JsonOutputParser()

final_decision = decision_chain.invoke({"user_message": user_message, "categorized_results": categorized_results, "research_info": full_searches, "strategy_analysis": strategy_analysis, "risk_assessment": risk_assessment, "sentiment_analysis": sentiment_analysis})

print(final_decision)

{'final_decision': {'decision': 'proceed', 'explanation': "Based on the analysis, I recommend proceeding with the long-term trading strategy for Apple Inc. (AAPL) stock. The strategy is suitable for the user's medium risk tolerance, and the potential profitability is moderate to high. The company has demonstrated a capacity for robust growth, and the overall sentiment is neutral, suggesting a balanced market without extreme buying or selling pressure. Although there are medium risks associated with AAPL stock, the benefits of the strategy outweigh the risks, and I conclude that the overall sentiment towards proceeding with the strategy is positive."}}


In [15]:
presentation_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are the Presentation Agent, an expert in summarizing complex financial analyses to end users. Your task is to provide a detailed and clear explanation of the final decision on whether to proceed with the trading strategy for the given stock. This explanation should be long and thorough, starting with the final decision and explaining everything that led to this decision.

Given the USER_MESSAGE, CATEGORIZED_RESULTS which includes STOCK_TICKER, RISK_TOLERANCE, and TRADING_STRATEGY, RESEARCH_INFO, STRATEGY_ANALYSIS, RISK_ASSESSMENT, SENTIMENT_ANALYSIS, and FINAL_DECISION, create a comprehensive summary.

Your summary should:
- Start with the final decision that is stated in FINAL_DECISION.
- Explain the reasoning behind this decision in detail, referencing specific findings from the research information, strategy analysis, risk assessment, and sentiment analysis.
- Ensure that the explanation is long and thorough, providing a clear understanding of the entire decision-making process.

Return your findings as a JSON object with a single key 'summary' containing your explanation. Do not include any preamble or explanation apart from this JSON object.<|eot_id|><|start_header_id|>user<|end_header_id|>

USER_MESSAGE: {user_message}

CATEGORIZED_RESULTS: {categorized_results}

RESEARCH_INFO: {research_info}

STRATEGY_ANALYSIS: {strategy_analysis}

RISK_ASSESSMENT: {risk_assessment}

SENTIMENT_ANALYSIS: {sentiment_analysis}

FINAL_DECISION: {final_decision}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["user_message","categorized_results","research_info","strategy_analysis","risk_assessment", "sentiment_analysis","final_decision"],
)

presentation_chain = presentation_prompt | GROQ_LLM | JsonOutputParser()

summary = presentation_chain.invoke({"user_message": user_message, "categorized_results": categorized_results, "research_info": full_searches, "strategy_analysis": strategy_analysis, "risk_assessment": risk_assessment, "sentiment_analysis": sentiment_analysis, "final_decision":final_decision})

print(summary)

{'summary': "Based on the analysis, I recommend proceeding with the long-term trading strategy for Apple Inc. (AAPL) stock. The strategy is suitable for the user's medium risk tolerance, and the potential profitability is moderate to high. The company has demonstrated a capacity for robust growth, and the overall sentiment is neutral, suggesting a balanced market without extreme buying or selling pressure. Although there are medium risks associated with AAPL stock, the benefits of the strategy outweigh the risks, and I conclude that the overall sentiment towards proceeding with the strategy is positive.\n\nThe research information indicates that AAPL has historically demonstrated a capacity for robust growth, with a record-breaking quarter for iPhone sales and firm performance in the Services sector. The company has also shown a strategic emphasis on investment in research and development expenses. The 30 analysts with 12-month price forecasts for Apple stock have an average target of 

In [ ]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    user_message : str
    categorized_results : str
    research_info : List[str]
    strategy_analysis : str
    risk_assessment : str
    sentiment_analysis : str
    final_decision : str
    summary : str
    num_steps : int

In [ ]:
def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
  """
  with open(f"{filename}.md", "w") as f:
    f.write(content)


In [ ]:
def categorize_message(state):
    """take the initial message and categorize it"""
    print("---CATEGORIZING INITIAL USER MESSAGE---")
    user_message = state['user_message']
    num_steps = int(state['num_steps'])
    num_steps += 1

    print(user_message)
    categorized_results = json_generator.invoke({"user_message": user_message})
    print(categorized_results['categorized_results'])

    # save to local disk
    #write_markdown_file(categorized_results, "message_category_results")

    return {"categorized_results": categorized_results['categorized_results'], "num_steps":num_steps}

In [ ]:
def research_info_search(state):
    print("---RESEARCH INFO SEARCHING---")
    user_message = state['user_message']
    categorized_results = state['categorized_results']
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Web search
    keywords = search_keyword_chain.invoke({"user_message": user_message, "categorized_results": categorized_results})
    # print(keywords)
    full_searches = []
    for keyword in keywords['search_prompts']:
        print(keyword)
        temp_docs = web_search_tool.invoke({"query": keyword})
        web_results = "\n".join([d["content"] for d in temp_docs])
        web_results = Document(page_content=web_results)
        if full_searches is not None:
            full_searches.append(web_results)
        else:
            full_searches = [web_results]
    print(full_searches)
    print(type(full_searches))
    # write_markdown_file(full_searches, "research_info")
    return {"research_info": full_searches, "num_steps":num_steps}

In [ ]:
def strategy_writer(state):
    print("---STRATEGY WRITER---")
    ## Get the state
    user_message = state["user_message"]
    categorized_results = state["categorized_results"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    print(user_message)
    print(categorized_results)
    print(research_info)

    strategy_analysis = strategy_chain.invoke({"user_message": user_message, "categorized_results": categorized_results, "research_info": research_info})
    print(strategy_analysis['strategy_analysis'])

    #write_markdown_file(analysis, "strategy_analysis")

    return {"strategy_analysis": strategy_analysis['strategy_analysis'], "num_steps":num_steps}

In [ ]:
def assess_risks(state):
    print("---RISK ASSESSER---")
    ## Get the state
    user_message = state["user_message"]
    categorized_results = state["categorized_results"]
    research_info = state["research_info"]
    strategy_analysis = state["strategy_analysis"]
    num_steps = state['num_steps']
    num_steps += 1

    print(user_message)
    print(categorized_results)
    print(research_info)
    print(strategy_analysis)

    risk_assessment = risk_assessment_chain.invoke({"user_message": user_message, "categorized_results": categorized_results, "research_info": research_info, "strategy_analysis": strategy_analysis})
    print(risk_assessment['risk_assessment'])

    #write_markdown_file(str(risk_assessment), "risk_assessment")
    return {"risk_assessment": risk_assessment['risk_assessment'], "num_steps":num_steps}

In [ ]:
def analyse_sentiment(state):
    print("---ANALYSE SENTIMENT ---")
    ## Get the state
    user_message = state["user_message"]
    categorized_results = state["categorized_results"]
    research_info = state["research_info"]
    strategy_analysis = state["strategy_analysis"]
    risk_assessment = state["risk_assessment"]
    num_steps = state['num_steps']
    num_steps += 1

    print(user_message)
    print(categorized_results)
    print(research_info)
    print(strategy_analysis)
    print(risk_assessment)

    sentiment_analysis = sentiment_chain.invoke({"user_message": user_message,
                                                 "categorized_results": categorized_results,
                                                 "research_info": research_info,
                                                 "strategy_analysis": strategy_analysis,
                                                 "risk_assessment": risk_assessment}
                                                )
    print(sentiment_analysis['sentiment_analysis'])

    #write_markdown_file(str(sentiment_analysis), "sentiment_analysis")
    return {"sentiment_analysis": sentiment_analysis['sentiment_analysis'], "num_steps":num_steps}

In [ ]:
def decide_final(state):
    print("---DECIDE FINAL ---")
    ## Get the state
    user_message = state["user_message"]
    categorized_results = state["categorized_results"]
    research_info = state["research_info"]
    strategy_analysis = state["strategy_analysis"]
    risk_assessment = state["risk_assessment"]
    sentiment_analysis = state["sentiment_analysis"]
    num_steps = state['num_steps']
    num_steps += 1

    print(user_message)
    print(categorized_results)
    print(research_info)
    print(strategy_analysis)
    print(risk_assessment)
    print(sentiment_analysis)

    final_decision = decision_chain.invoke({"user_message": user_message,
                                            "categorized_results": categorized_results,
                                            "research_info": research_info,
                                            "strategy_analysis": strategy_analysis,
                                            "risk_assessment": risk_assessment,
                                            "sentiment_analysis": sentiment_analysis}
                                           )
    print(final_decision['final_decision'])

    #write_markdown_file(str(final_decision), "final_decision")
    return {"final_decision": final_decision['final_decision'], "num_steps":num_steps}

In [ ]:
def make_presentation(state):
    print("---MAKE PRESENTATION---")
    ## Get the state
    user_message = state["user_message"]
    categorized_results = state["categorized_results"]
    research_info = state["research_info"]
    strategy_analysis = state["strategy_analysis"]
    risk_assessment = state["risk_assessment"]
    sentiment_analysis = state["sentiment_analysis"]
    final_decision = state['final_decision']
    num_steps = state['num_steps']
    num_steps += 1

    print(user_message)
    print(categorized_results)
    print(research_info)
    print(strategy_analysis)
    print(risk_assessment)
    print(sentiment_analysis)
    print(final_decision)

    summary = presentation_chain.invoke({"user_message": user_message,
                                         "categorized_results": categorized_results,
                                         "research_info": research_info,
                                         "strategy_analysis": strategy_analysis,
                                         "risk_assessment": risk_assessment,
                                         "sentiment_analysis": sentiment_analysis,
                                         "final_decision":final_decision}
                                        )
    print(summary['summary'])

    #write_markdown_file(str(summary), "summary")
    return {"summary": summary['summary'], "num_steps":num_steps}

In [ ]:
def state_printer(state):
    """print the state"""
    print("---STATE PRINTER---")
    print(f"User Message: {state['user_message']} \n" )
    print(f"Categorized Results: {state['categorized_results']} \n")
    print(f"Research Info: {state['research_info']} \n" )
    print(f"Strategy Analysis: {state['strategy_analysis']} \n" )
    print(f"Risk Assessment: {state['risk_assessment']} \n")
    print(f"Sentiment Analysis: {state['sentiment_analysis']} \n")
    print(f"Final Decision: {state['final_decision']} \n")
    print(f"Summary: {state['summary']} \n")
    print(f"Num Steps: {state['num_steps']} \n")
    return

In [ ]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("categorize_message", categorize_message)
workflow.add_node("research_info_search", research_info_search) # web search
workflow.add_node("strategy_writer", strategy_writer)
workflow.add_node("assess_risks", assess_risks)
workflow.add_node("analyse_sentiment", analyse_sentiment)
workflow.add_node("decide_final", decide_final)
workflow.add_node("make_presentation", make_presentation)

In [ ]:
workflow.set_entry_point("categorize_message")
workflow.add_edge("categorize_message", "research_info_search")
workflow.add_edge("research_info_search", "strategy_writer")
workflow.add_edge("strategy_writer", "assess_risks")
workflow.add_edge("assess_risks", "analyse_sentiment")
workflow.add_edge("analyse_sentiment", "decide_final")
workflow.add_edge("decide_final", "make_presentation")
workflow.add_edge("make_presentation", END)

In [ ]:
# Compile
app = workflow.compile()

In [ ]:
EMAIL = """I am interested in trading apple stock. I prefer day trading, I don't want to lose my money if it is risky to enter the market at the moment."""
# EMAIL = """I am interested in trading google stock. I prefer long-term investment, I am open to take risks in short term if it is going to end in my benefit in the long run."""
# EMAIL = """I am considering investing in Tesla stock. I have a medium risk tolerance and prefer a swing trading strategy. I'm looking to capitalize on short-term price movements and hold the stock for a few weeks to a few months."""
# EMAIL = """I want to invest all my savings into penny stocks because I heard they can give high returns very quickly. I don't really have any experience with trading and I'm okay with taking big risks to try and make a lot of money fast."""
# EMAIL = """I want to invest in a stock. My risk tolerance is also medium and my trading strategy is swing trading. Could you analyze if this investment is suitable for me and provide any stock recommendations?"""

In [ ]:
# run the agent
inputs = {"user_message": EMAIL,"research_info": None, "num_steps":0}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")

---CATEGORIZING INITIAL USER MESSAGE---
I am interested in trading apple stock. I prefer day trading, I don't want to lose my money if it is risky to enter the market at the moment.
{'stock_ticker': 'AAPL', 'risk_tolerance': 'Low', 'trading_strategy': 'Day Trading'}
'Finished running: categorize_message:'
---RESEARCH INFO SEARCHING---
AAPL current stock price and historical price trends
AAPL day trading market trend predictions
Financial analysts' analysis and predictions for AAPL stock
Recent news and developments affecting AAPL stock
Risk management strategies for low-risk day trading AAPL stock
[Document(page_content='Get the latest Apple Inc (AAPL) real-time quote, historical performance, charts, and other financial information to help you make more informed trading and investment decisions.\nHistorical daily share price chart and data for Apple since 1980 adjusted for splits and dividends. The latest closing stock price for Apple as of May 24, 2024 is 189.98.. The all-time high Ap

In [ ]:
print(output['make_presentation']['summary'])

Based on the comprehensive analysis of the specified trading strategy, day trading, for Apple Inc. (AAPL) stock, the final decision is to not proceed with the strategy due to the high risks involved. 

The research information highlights AAPL's historical performance, including its 52-week high and low stock prices, indicating potential fluctuations in the stock price. The stock's exposure to China, which is dwindling, and the end of its credit card partnership with Goldman Sachs may also impact the stock price. 

The strategy analysis reveals that the specified trading strategy, day trading, is not suitable for users with low risk tolerance. The projected price of 200.37 is higher than the current price of 189.98, but the risks involved are too high for low-risk tolerance users. 

The risk assessment identifies specific risks, including market volatility, potential loss, historical risk factors, and external factors. The potential financial impact of these risks is substantial, with a